In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [4]:
data = pd.read_csv('data/pristavki.csv', header=None, names=['text'])

In [62]:
data.text[50:100]

50    Ps move с камерой/\nПродаю из-за не надобности/\nРаботает отлично/\nТорга нет                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [115]:
from yargy import Parser, rule, or_, and_
from yargy.predicates import in_, in_caseless, gte, lte
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

In [40]:
GameConsole = fact(
    'GameConsole',
    ['name', 'model', 'version']
)

In [41]:
PS_VERSIONS = {
    'super slim': 'SuperSlim',
    'superslim': 'SuperSlim',
    'slim': 'Slim',
    'fat': 'Fat',
    'pro': 'PRO',
    'vita': 'VITA'
}

In [80]:
PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS', 'плойка']).interpretation(GameConsole.name.const("Playstation")),
    in_('1234').interpretation(GameConsole.model.normalized()),
    caseless_pipeline(PS_VERSIONS).interpretation(GameConsole.version.normalized().custom(PS_VERSIONS.get)).optional()
)
XBOX = rule(
    morph_pipeline(['Xbox', 'X box', 'иксбокс', 'бокс']).interpretation(GameConsole.name.const("Xbox")),
    morph_pipeline(['1', '360', 'One', 'ван']).interpretation(GameConsole.model),
    in_caseless('sxe').interpretation(GameConsole.version.normalized()).optional()
)

In [48]:
GAME_CONSOLE = or_(PS, XBOX).interpretation(GameConsole)

In [11]:
Game = fact(
    'Game',
    ['title', 'platform']
)

In [125]:
GAME_RULES = (rule(
    morph_pipeline(['Battlefield', 'Battle field', 'бателфилд', 'бателфиелд']).interpretation(Game.title.const("Battlefield")),
),
rule(
    morph_pipeline(['God Of War', 'Год оф вар']).interpretation(Game.title.const('God Of War')),
),
rule(
    morph_pipeline(['Need For Speed', 'nfs']).interpretation(Game.title.const('Need For Speed'))
),
rule(
    morph_pipeline(['Dark Souls', 'дарк соулс']).interpretation(Game.title.const('Dark Souls'))
),
rule(
    morph_pipeline(['FIFA', 'фифа']).interpretation(Game.title.const('FIFA')),
    and_(gte(10), lte(22))
),
rule(
    morph_pipeline(['GTA', 'GTA', 'гта']).interpretation(Game.title.const("GTA")),
    gte(2)
),
rule(
    morph_pipeline(['Far Cry', 'FarCry', 'фар край']).interpretation(Game.title.const("Far Cry")),
    morph_pipeline(['V', '5', '4'])
),
rule(
    morph_pipeline(['Watch Dogs', 'вотч догс']).interpretation(Game.title.const('Watch Dogs')),
    in_caseless('12')
))

In [126]:
GAME = rule(
    or_(*GAME_RULES),
    GAME_CONSOLE.interpretation(Game.platform).optional()
).interpretation(Game)

In [127]:
parser = Parser(GAME)

In [131]:
matches = []

for sent in data.text:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [132]:
matches

[Game(title='FIFA',
      platform=None), Game(title='GTA',
      platform=None), Game(title='GTA',
      platform=None), Game(title='FIFA',
      platform=None), Game(title='GTA',
      platform=None), Game(title='Battlefield',
      platform=None), Game(title='Need For Speed',
      platform=None), Game(title='FIFA',
      platform=None), Game(title='FIFA',
      platform=None), Game(title='GTA',
      platform=None), Game(title='Battlefield',
      platform=None), Game(title='Battlefield',
      platform=None), Game(title='Battlefield',
      platform=None), Game(title='Need For Speed',
      platform=None), Game(title='Battlefield',
      platform=None), Game(title='GTA',
      platform=None), Game(title='Need For Speed',
      platform=None), Game(title='God Of War',
      platform=None), Game(title='Battlefield',
      platform=None), Game(title='Need For Speed',
      platform=None), Game(title='Battlefield',
      platform=None), Game(title='Far Cry',
      platform=None), Game

In [134]:
len(matches)

1721